In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('corpus/lover_lyrics.csv', encoding = 'latin-1')

In [3]:
df.head()

,ï»¿album_name,track_title,track_n,lyric,line
0,Lover,I Forgot That You Existed,1,How many days did I spend thinking,1
1,Lover,I Forgot That You Existed,1,"'Bout how you did me wrong, wrong, wrong?",2
2,Lover,I Forgot That You Existed,1,Lived inâ theâ shade you wereâ throwing,3
3,Lover,I Forgot That You Existed,1,"'Til all of my sunshine wasâ gone, gone, gone",4
4,Lover,I Forgot That You Existed,1,And I couldn't get away from ya,5


In [4]:
songs = df.groupby('track_title').agg({'lyric': lambda x: " ".join(x)}).reset_index()

In [5]:
pd.options.display.max_colwidth = 5000

In [6]:
songs.head()

,track_title,lyric
0,Afterglow,"I blew things out of proportion, now you're blue Put you in jail for something youâ didnâtâ do I pinned yourâ hands behind your back, oh Thought Iâ had reason to attack, but no Fighting with a true love is boxing with no gloves Chemistry 'til it blows up, 'til thereâs no us Why'd I have to break what I love so much? It's on your face, and I'm to blame, I need to say Hey, it's all me, in my head I'm the one who burned us down But it's not what I meant Sorry that I hurt you I don't wanna do, I donât wanna do this to you (Ooh) I donât wanna lose, I don't wanna lose this with you (Ooh) I need to say, hey, itâs all me, just don't go Meet me in the afterglow It's so excruciating to see you low Just wanna lift you up and not let you go This ultraviolet morning light below Tells me this love is worth the fight, oh I lived like an island, punished you with silence Went off like sirens, just crying Why'd I have to break what I love so much? Itâs on your face, don't walk away, I need to say Hey, it's all me, in my head I'm the one who burned us down But it's not what I meant I'm sorry that I hurt you I don't wanna do, I don't wanna do this to you (Ooh) I don't wanna lose, I don't wanna lose this with you (Ooh) I need to say, hey, it's all me, just don't go Meet me in the afterglow Tell me that you're still mine Tell me that we'll be just fine Even when I lose my mind I need to say Tell me that it's not my fault Tell me that I'm all you want Even when I break your heart I need to say Hey, it's all me, in my head I'm the one who burned us down But it's not what I meant Sorry that I hurt you I don't wanna do, I don't wanna do this to you (Ooh) I don't wanna lose, I don't wanna lose this with you (Ooh) I need to say, hey, it's all me, just don't go Meet me in the afterglow"
1,Cornelia Street,"We were in the backseat Drunk on something stronger than the drinks in the bar ""I rentâ aâ place on Corneliaâ Street"" I say casually in the car Weâ were a fresh page on the desk Filling in the blanks as we go As if the street lights pointed in an arrow head Leading us home And I hope I never lose you, hope it never ends I'd never walk Cornelia Street again That's the kinda heartbreak time could never mend I'd never walk Cornelia Street again And baby, I get mystified by how this city screams your name And baby, I'm so terrified of if you ever walk away I'd never walk Cornelia Street again I'd never walk Cornelia Street again Windows flung right open, autumn air Jacket 'round my shoulders is yours We bless the rains on Cornelia Street Memorize the creaks in the floor Back when we were card sharks, playing games I thought you were leading me on I packed my bags, left Cornelia Street Before you even knew I was gone But then you called,Â showed your hand I turned around before I hit the tunnel Sat on the roof, you and I I hope I never lose you, hope it never ends I'd never walk Cornelia Street again That's the kinda heartbreak time could never mend I'd never walk Cornelia Street again And baby, I get mystified by how this city screams your name And baby, I'm so terrified of if you ever walk away I'd never walk Cornelia Street again I'd never walk Cornelia Street again You hold my hand on the street Walk me back to that apartment Years ago, we were just inside Barefoot in the kitchen Sacred new beginnings That became my religion, listen I hope I never lose you I'd never walk Cornelia Street again Oh, never again And baby, I get mystified by how this city screams your name And baby, I'm so terrified of if you ever walk away I'd never walk Cornelia Street again I'd never walk Cornelia Street again I don't wanna lose you (Hope it never ends) I'd never walk Cornelia Street again I don't wanna lose you (Yeah) ""I rent a place on Cornelia Street"" I say casually in the car"
2,Cruel Summer,"(Yeah, yeah, yeah, yeah) Fever dream high in the quiet of the night You know that I caught it (Oh yeah, you're right, I wan

In [7]:
len(songs)

18

In [8]:
import sklearn

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/anita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop_words = stopwords.words('english')

In [12]:
stopwords = [ "darling",
    "ooh",
    "yeah",
    "whoa",
    "wanna",
    "iâ",
    "okay",
    "cause",
    "get",
    "mad"]

In [13]:
vectorizer = TfidfVectorizer(stop_words = stop_words, min_df = 0.1)

In [14]:
tfidf = vectorizer.fit_transform(songs['lyric'])

In [15]:
from sklearn.decomposition import NMF

In [16]:
nmf = NMF(n_components = 6)

In [17]:
topic_values = nmf.fit_transform(tfidf)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [18]:
for topic_num, topic in enumerate(nmf.components_):
    message = "Topic #{}: ".format(topic_num + 1)
    message += " ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11 :-1]])
    print(message)

Topic #1: want right see bless think one could stay darling got
Topic #2: ooh nice soon better fun one baby get like yeah
Topic #3: oh need wanna summer boy yeah whoa love know cruel
Topic #4: still love even god enough get see look touch faith
Topic #5: man would coming running iâ okay bad cause get mad
Topic #6: street walk never take lover ever lose go place always


In [19]:
topic_labels = ['relationships', 'excitement', 'desire', 'devotion', 'anger', 'nostalgia']

In [20]:
df_topics = pd.DataFrame(topic_values, columns = topic_labels)
df_topics

,relationships,excitement,desire,devotion,anger,nostalgia
0,0.000000,0.186601,0.315437,0.157303,0.000000,0.133984
1,0.000000,0.011577,0.000000,0.000000,0.000000,0.865579
2,0.053848,0.129510,0.558704,0.000000,0.011026,0.000000
3,0.162310,0.000000,0.000000,0.435835,0.000000,0.034136
4,0.000000,0.000000,0.000000,0.679519,0.000000,0.034866
5,0.000000,0.000000,0.000000,0.749000,0.000000,0.000000
6,0.006543,0.207714,0.000000,0.219182,0.210259,0.000000
7,0.369725,0.000000,0.024022,0.000000,0.039398,0.000000
8,0.000000,0.759478,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.077693,0.476849,0.204562,0.000000,0.008022


In [21]:
songs = songs.join(df_topics) #['relationships', 'excitement', 'desire', 'devotion', 'anger', 'nostalgia']

In [22]:
songs.loc[songs['relationships'] >= 0.1, 'relationships'] = 1
songs.loc[songs['excitement'] >= 0.1, 'excitement'] = 1
songs.loc[songs['desire'] >= 0.1, 'desire'] = 1
songs.loc[songs['devotion'] >= 0.1, 'devotion'] = 1
songs.loc[songs['anger'] >= 0.1, 'anger'] = 1
songs.loc[songs['nostalgia'] >= 0.1, 'nostalgia'] = 1

In [23]:
songs.loc[songs['relationships'] < 0.1, 'relationships'] = 0
songs.loc[songs['excitement'] < 0.1, 'excitement'] = 0
songs.loc[songs['desire'] < 0.1, 'desire'] = 0
songs.loc[songs['devotion'] < 0.1, 'devotion'] = 0
songs.loc[songs['anger'] < 0.1, 'anger'] = 0
songs.loc[songs['nostalgia'] < 0.1, 'nostalgia'] = 0

In [24]:
songs_csv = songs.to_csv('topics/lover_topics.csv')